In [ ]:
!pip install --upgrade keras
!pip install --upgrade keras tensorflow
import os
import math
import numpy as np
import librosa
import json
import pandas as pd
from termcolor import colored
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
!pip install tensorboard
!pip install visualkeras
import visualkeras
from keras.utils import plot_model
from datetime import datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn import svm
from sklearn.svm import SVC
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from IPython.display import Audio
from scipy.io import wavfile
import scipy
import soundfile as sf
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report
import random

from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.7/347.7 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.4.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 3.0 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling m

In [ ]:
import os
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

def create_mel_spectrogram(audio_file, output_dir, sr=16000, n_fft=2048, hop_length=256, n_mels=128):
    y, sr = librosa.load(audio_file, sr=sr)

    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
    mel_spec_db = librosa.amplitude_to_db(mel_spec, ref=np.max)

    plt.figure(figsize=(10, 4))
    librosa.display.specshow(mel_spec_db, sr=sr, hop_length=hop_length, x_axis='time', y_axis='mel')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel Spectrogram')
    plt.tight_layout()

    filename = os.path.basename(audio_file).split('.')[0] + '_mel.png'
    save_path = os.path.join(output_dir, filename)
    plt.savefig(save_path)
    plt.close()

    print(f'Mel spectrogram saved as {filename}')

if __name__ == '__main__':
    file_counter = 0

    audio_dir = '/content/drive/MyDrive/Colab Notebooks/TAMUCC REU 2024/DS1/Al-Emadi_Standardized_Audio/Test/Spark/'
    output_dir = '/content/drive/MyDrive/UnknownDrone_256HL'

    os.makedirs(output_dir, exist_ok=True)

    for file in os.listdir(audio_dir):
        if file.endswith('.wav') or file.endswith('.mp3'):
            audio_file = os.path.join(audio_dir, file)
            create_mel_spectrogram(audio_file, output_dir)
            file_counter+=1

    print(f"Total files: {file_counter}")

Mel spectrogram saved as Spark_Test_File1_1_mel.png
Mel spectrogram saved as Spark_Test_File1_7_mel.png
Mel spectrogram saved as Spark_Test_File2_3_mel.png
Mel spectrogram saved as Spark_Test_File2_6_mel.png
Mel spectrogram saved as Spark_Test_File2_8_mel.png
Mel spectrogram saved as Spark_Test_File2_14_mel.png
Mel spectrogram saved as Spark_Test_File2_19_mel.png
Mel spectrogram saved as Spark_Test_File2_21_mel.png
Mel spectrogram saved as Spark_Test_File3_3_mel.png
Mel spectrogram saved as Spark_Test_File3_9_mel.png
Mel spectrogram saved as Spark_Test_File1_1_PS-2_mel.png
Mel spectrogram saved as Spark_Test_File1_7_PS-2_mel.png
Mel spectrogram saved as Spark_Test_File2_3_PS-2_mel.png
Mel spectrogram saved as Spark_Test_File2_6_PS-2_mel.png
Mel spectrogram saved as Spark_Test_File2_8_PS-2_mel.png
Mel spectrogram saved as Spark_Test_File2_14_PS-2_mel.png
Mel spectrogram saved as Spark_Test_File2_19_PS-2_mel.png
Mel spectrogram saved as Spark_Test_File2_21_PS-2_mel.png
Mel spectrogram sa

In [ ]:
import os

# Specify the main directory path
main_directory = '/content/drive/MyDrive/UnknownDrone_256HL'

# Loop through all subdirectories and count files
for root, dirs, files in os.walk(main_directory):
    print(f"Directory: {root}, Number of files: {len(files)}")

Directory: /content/drive/MyDrive/UnknownDrone_256HL, Number of files: 250


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = '/content/drive/MyDrive/Colab Notebooks/TAMUCC REU 2024/DS1/Al-Emadi_Standardized_Unknown_Spectrograms/Train/'
test_dir = '/content/drive/MyDrive/Colab Notebooks/TAMUCC REU 2024/DS1/Al-Emadi_Standardized_Unknown_Spectrograms/Test/'

num_classes = 7
height = 128
n_fft = 2048
hop_length = 512
sr = 16000
T = 1

hop_size_sec = hop_length / sr
audio_length_samples = T * sr

width = 1 + int((audio_length_samples - n_fft) / hop_length)

channels = 3

train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator()

batch_size = 32

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical')

model = tf.keras.Sequential()

# Convolutional layers
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(height, width, channels)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

# Flatten layer to feed into fully connected layers
model.add(tf.keras.layers.Flatten())

# Dense layers
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

epochs = 200
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    train_generator.reset()

    model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=1,
        verbose=1)

loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

model.save('el-amadi_cnn_standardized_unknown.keras')

Found 3088 images belonging to 7 classes.
Found 768 images belonging to 7 classes.
Epoch 1/200
97/97 ━━━━━━━━━━━━━━━━━━━━ 191s 2s/step - accuracy: 0.3916 - loss: 12.4379
Epoch 2/200
97/97 ━━━━━━━━━━━━━━━━━━━━ 74s 714ms/step - accuracy: 0.6277 - loss: 1.0954
Epoch 3/200
97/97 ━━━━━━━━━━━━━━━━━━━━ 82s 804ms/step - accuracy: 0.6764 - loss: 0.9123
Epoch 4/200
97/97 ━━━━━━━━━━━━━━━━━━━━ 78s 753ms/step - accuracy: 0.7335 - loss: 0.7897
Epoch 5/200
97/97 ━━━━━━━━━━━━━━━━━━━━ 74s 714ms/step - accuracy: 0.7197 - loss: 0.7998
Epoch 6/200
97/97 ━━━━━━━━━━━━━━━━━━━━ 75s 716ms/step - accuracy: 0.7655 - loss: 0.6582
Epoch 7/200
97/97 ━━━━━━━━━━━━━━━━━━━━ 75s 722ms/step - accuracy: 0.7803 - loss: 0.6050
Epoch 8/200
97/97 ━━━━━━━━━━━━━━━━━━━━ 74s 717ms/step - accuracy: 0.8141 - loss: 0.5487
Epoch 9/200
97/97 ━━━━━━━━━━━━━━━━━━━━ 73s 706ms/step - accuracy: 0.8137 - loss: 0.5149
Epoch 10/200
97/97 ━━━━━━━━━━━━━━━━━━━━ 81s 781ms/step - accuracy: 0.8240 - loss: 0.4808
Epoch 11/200
97/97 ━━━━━━━━━━━━━━━━━━

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator

num_classes = 7
height = 128
n_fft = 2048
hop_length = 512
sr = 16000
T = 1

hop_size_sec = hop_length / sr
audio_length_samples = T * sr

width = 1 + int((audio_length_samples - n_fft) / hop_length)

channels = 3

batch_size = 32

model = tf.keras.models.load_model('/content/el-amadi_cnn_standardized_unknown.keras')

test_dir = '/content/drive/MyDrive/Colab Notebooks/TAMUCC REU 2024/DS1/Al-Emadi_Standardized_Unknown_Spectrograms/Test/'

test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Get true class labels
true_classes = test_generator.classes

# Compute confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("\nConfusion Matrix:")
print(conf_matrix)

# Get class labels
class_labels = list(test_generator.class_indices.keys())

# Calculate accuracy for each class
class_accuracy = {}
for i, class_label in enumerate(class_labels):
    correct_indices = np.where(true_classes == i)[0]
    correct_predictions = np.sum(predicted_classes[correct_indices] == i)
    total_samples = len(correct_indices)
    class_accuracy[class_label] = correct_predictions / total_samples if total_samples > 0 else 0.0

# Print accuracy for each class
print("\nAccuracy for each class:")
for class_label in class_labels:
    print(f"{class_label}: {class_accuracy[class_label]:.4f}")

# Print classification report (includes precision, recall, F1-score)
print("\nClassification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

Found 768 images belonging to 7 classes.
24/24 ━━━━━━━━━━━━━━━━━━━━ 16s 612ms/step - accuracy: 0.9598 - loss: 0.2426
Test Loss: 0.21118313074111938
Test Accuracy: 0.9622395634651184
24/24 ━━━━━━━━━━━━━━━━━━━━ 15s 611ms/step

Confusion Matrix:
[[102   0   1   0   0   1   0]
 [  0  52   0   0   0   0   0]
 [  3   0  92   0   0   9   0]
 [  0   0   0  50   0   0   0]
 [  0   0   0   5  42   3   0]
 [  3   0   4   0   0 350   0]
 [  0   0   0   0   0   0  51]]

Accuracy for each class:
Bebop: 0.9808
Crazyflie: 1.0000
Mambo: 0.8846
Matrice100: 1.0000
Mavic: 0.8400
NonDrone: 0.9804
Phantom: 1.0000

Classification Report:
              precision    recall  f1-score   support

       Bebop       0.94      0.98      0.96       104
   Crazyflie       1.00      1.00      1.00        52
       Mambo       0.95      0.88      0.92       104
  Matrice100       0.91      1.00      0.95        50
       Mavic       1.00      0.84      0.91        50
    NonDrone       0.96      0.98      0.97       35

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# Assume you have a trained model already loaded
model = tf.keras.models.load_model('/content/el-amadi_cnn_standardized_unknown.keras')  # Load your model here

# Define your classes
classes = ["Bebop", "Crazyflie", "Mambo", "Matrice100", "Mavic", "Phantom"]

# Define a confidence threshold below which you classify as "Unknown Drone"
confidence_threshold = 0.6

def preprocess_image(img_path):
    # Load and preprocess the image
    num_classes = 7
    height = 128
    n_fft = 2048
    hop_length = 512
    sr = 16000
    T = 1

    hop_size_sec = hop_length / sr
    audio_length_samples = T * sr

    width = 1 + int((audio_length_samples - n_fft) / hop_length)

    channels = 3

    img = image.load_img(img_path, target_size=(height, width))  # Assuming input size expected by your model
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Normalize pixel values to [0, 1]
    return img_array

def classify_unknown_drones_in_directory(directory):
    # Ensure directory exists
    if not os.path.exists(directory):
        print(f"Directory '{directory}' does not exist.")
        return

    file_counter = 0
    unknown_counter = 0

    # Iterate through files in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
            # Construct full file path
            file_counter+=1
            img_path = os.path.join(directory, filename)

            # Preprocess the image
            img = preprocess_image(img_path)

            # Predict probabilities for each class
            predicted_probs = model.predict(img)[0]

            # Get the predicted class and its confidence score
            predicted_class_index = np.argmax(predicted_probs)
            predicted_class = classes[predicted_class_index]
            confidence_score = predicted_probs[predicted_class_index]

            # Determine classification result
            if confidence_score >= confidence_threshold:
                print(f"Image '{filename}' - Predicted class: {predicted_class}")
            else:
                print(f"Image '{filename}' - Unknown Drone")
                unknown_counter+=1

    print(f"Total files: {file_counter}")
    print(f"Unknown drones: {unknown_counter}")

# Example usage:
directory_path = "/content/drive/MyDrive/Colab Notebooks/TAMUCC REU 2024/DS1/Al-Emadi_Standardized_Unknown_Spectrograms/Test/UnknownDrone"
classify_unknown_drones_in_directory(directory_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 504ms/step
Image 'Spark_Test_File1_7_PS-2_mel.png' - Predicted class: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
Image 'Spark_Test_File3_1_PS-1_mel.png' - Predicted class: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
Image 'Spark_Test_File2_1_PS+2_mel.png' - Predicted class: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Image 'Spark_Test_File2_10_PS-2_mel.png' - Predicted class: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Image 'Spark_Test_File2_0_PS-1_mel.png' - Predicted class: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Image 'Spark_Test_File1_0_PS-2_mel.png' - Predicted class: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Image 'Spark_Test_File2_20_PS+1_mel.png' - Predicted class: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Image 'Spark_Test_File2_6_PS+2_mel.png' - Predicted class: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Image 'Spark_Test_File2_17_PS-2_mel.png' - Predicted class: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Image 'Spar